#Comparing the LLM Tokenizers#
This will show us that newer toeknizers have changed their behaviourto improve model performance. And speciallized models need specialized charachgters.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

This will allow us to see how each tokenizer deals with a number of different kinds of tokens:
* Capitalization.
* Languages other than English.
* Emojis.
* Programming code with keywords and whitespaces often used for indentation (in languages like Python for example).
* Numbers and digits.
* Special tokens. These are unique tokens that have a role other than representing text. They include tokens that indicate the beginning of the text, or the end of the text (which is the way the model signals to the system that it has completed this generation), or other functions as we'll see.

In [4]:
text = """
English and CAPITALIZATION
🎵 鸟 هاااااای
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

#BERT uncased

In [5]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] [UNK] ه ##ا ##ا ##ا ##ا ##ا ##ا ##ی show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

We noticed that:
* The newline breaks are gone, which makes the model blind to information encoded in newlines (e.g., a chat log when each turn is in a new line).
* All the text is in lowercase.
* The word "capitalization" is encoded as two subtokens: capital ##ization.
* The ## characters are used to indicate this token is a partial token connected to the token that precedes it. This is also a method to indicate where the spaces are, as it is assumed tokens without ## in front have a space before them.
* The emoji and Chinese characters are gone and replaced with the [UNK] special token indicating an "unknown token."

#BERT cased

In [7]:
show_tokens(text, "bert-base-cased")

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] ه ##ا ##ا ##ا ##ا ##ا ##ا ##ی show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

Notice how "CAPITALIZATION" is now represented as eight tokens: CA ##PI ##TA ##L ##I ##Z ##AT ##ION.

Both BERT tokenizers wrap the input within a starting [CLS] token and a closing [SEP] token. [CLS] and [SEP] are utility tokens used to wrap the input text and they serve their own purposes. [CLS] stands for classification as it's a token used at times for sentence classification. [SEP] stands for separator, as it's used to separate sentences in some applications that require passing two sentences to a model.

#GPT-2

In [9]:
show_tokens(text, "gpt2")


 English  and  CAP ITAL IZ ATION 
 � � �  � � �  � � ا ا ا ا ا ا � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

* The newline breaks are represented in the tokenizer.
* Capitalization is preserved, and the word "CAPITALIZATION" is represented in four tokens.
* The鸟 characters are now represented by multiple tokens each. While we see these tokens printed as the character, they actually stand for different tokens. For example, the emoji is broken down into the tokens with token IDs 8582, 236, and 113. The tokenizer is successful in reconstructing the original character from these tokens. We can see that by printing tokenizer.decode([8582, 236, 113]), which prints out.
* The two tabs are represented as two tokens (token number 197 in that vocabu-lary) and the four spaces are represented as three tokens (number 220) with the final space being a part of the token for the closing quote character.

In [27]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
token_ids = tokenizer("🎵").input_ids
print('🎵')
print(token_ids)
show_tokens("🎵", "gpt2")
print('\n')
print('鸟')
token_ids = tokenizer("鸟").input_ids
print(token_ids)
show_tokens("鸟", "gpt2")
print('\n')
print('Printing two tabs')
token_ids = tokenizer("   ").input_ids
print(token_ids)
show_tokens("   ", "gpt2")
print('\n')
print('Printing four spaces')
token_ids = tokenizer("    ").input_ids
print(token_ids)
show_tokens("    ", "gpt2")

🎵
[8582, 236, 113]
� � � 

鸟
[165, 116, 253]
� � � 

Printing two tabs
[220, 220, 220]
      

Printing four spaces
[220, 220, 220, 220]
        

#Flan-T5

In [29]:
show_tokens(text, "google/flan-t5-small")

English and CA PI TAL IZ ATION  <unk>  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

The Flan-T5 family of models use the SentencePiece method. We notice the following:
* No newline or whitespace tokens; this would make it challenging for the model to work with code.
* The emoji and Chinese characters are both replaced by the <unk> token, making the model completely blind to them

#GPT4

In [32]:
# The official is `tiktoken` but this the same tokenizer on the HF platform
show_tokens(text, "Xenova/gpt-4")


 English  and  CAPITAL IZATION 
 � � �  � � �  ه ا ا ا ا ا ای 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

* Fill in the middle tokens. These three tokens enable the LLM to generate a completion given not only the text before it but also considering the text after it.
* The GPT-4 tokenizer behaves similarly to its ancestor, the GPT-2 tokenizer. Some differences are:
* It tokenizer represents the four spaces as a single token. In fact, it has a specific token for every sequence of whitespaces up to a list of 83 whitespaces.
* The Python keyword elif has its own token in GPT-4. Both this and the previ-ous point stem from the model's focus on code in addition to natural language.
* The GPT-4 tokenizer uses fewer tokens to represent most words. Examples here include "CAPITALIZATION" (two tokens versus four) and "tokens" (one token versus three).

#StarCode
Focouses on generating code

In [34]:
# You need to request access before being able to use this tokenizer
show_tokens(text, "bigcode/starcoder2-15b")


 English  and  CAPITAL IZATION 
 � � �   � �  ه ا ا ا ا ا ای 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

* A major difference here to everything we've seen so far is that each digit is assigned its own token (so 600 becomes 600). The hypothesis here is that this would lead to better representation of numbers and mathematics. In GPT-2, for example, the number 870 is represented as a single token. But 871 is represented as two tokens (8 and 71). You can intuitively see how that might be confusing to the model and how it represents numbers



* When representing code, managing the context is important. One file might make a function call to a function that is defined in a different file. So the model needs some way of being able to identify code that is in different files in the same code repository, while making a distinction between code in different repos.

#Galactica
An LLM for science

In [7]:
show_tokens(text, "facebook/galactica-1.3b")


 English  and  CAP ITAL IZATION 
 � � � �  � � �   ه ا ا ا ا ا ا ی 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

The Galactica tokenizer behaves similar to StarCoder2 in that it has code in mind. It also encodes whitespaces in the same way: assigning a single token to sequences of whitespace of different lengths. It differs in that it also does that for tabs, though. So from all the tokenizers we've seen so far, it's the only one that assigns a single token to the string made up of two tabs ('\t\t').

In [9]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � �  ه ا ا ا ا ا ا ی 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 